[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/ersip-sprint/blob/master/sprints/1/2%20-%20Load%20and%20display%20LiDAR%20raster.ipynb)

In [1]:
from pathlib import Path # Manage folders and files
import rasterio as rio # Reads and writes geospatial raster data
from matplotlib import pyplot as plt # Display plots and images
import numpy as np # Multi-dimensional arrays

# Load and display LiDAR raster

The LiDAR raster files are located in the `data` folder. The rasters are in single band `TIFF` image format.

In [2]:
data_dir = Path('data')
list(data_dir.glob('*.tif'))

[PosixPath('data/dsm_vox_50cm_tile_-7_1.tif'),
 PosixPath('data/dsm_vox_50cm_tile_-12_0.tif'),
 PosixPath('data/dsm_vox_50cm_tile_-11_0.tif')]

## 1. Load and display a raster file

- Open the raster file and read the 2D array (tip: you can type `?function` to display docstrings, e.g. `?rio.open` )
- Read the 2D array
- Display the 2D array as an image
    + what can we do to visualize more details?

In [3]:
# Your turn!

## 2. Visualize flat zones in the DSM

- Find the values of the flat zones
    + Sort the unique values in a list (or an array) of levels
- Display some flat zones
    + Display the flatzones of $i^{th}$ level

In [4]:
# Your turn!

## 3. Create a max-tree of the DSM

Building a max-tree from scratch is a challenging assignement 🙂

In [5]:
# If you have time, you can start work on a max-tree algorithm 